# Comparing Models Performance

In [1]:
!pip install rouge_score

## Importing modules

In [19]:
'''
# Get the scripts from remote source
#Run this if your working environment is Unix-like (Linux, Mac, etc.)
!git clone https://github.com/mlgomez0/Health_Therapist
!mv Health_Therapist/backend/ml_models/modules .
!mv Health_Therapist/backend/ml_models/datasets .
!mv Health_Therapist/backend/ml_models/LLMs_fine_tuning/phi3-qlora .
!move Health_Therapist\backend\ml_models\LLMs_fine_tuning\gpt_instruction_tuning .
!mv Health_Therapist/requirements.txt .
!rm -rf Health_Therapist
'''

fatal: destination path 'Health_Therapist' already exists and is not an empty directory.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# Get the scripts from remote source
#Run this if your working environment is windows
!git clone https://github.com/mlgomez0/Health_Therapist
!move Health_Therapist\backend\ml_models\modules .
!move Health_Therapist\backend\ml_models\datasets .
!move Health_Therapist\backend\ml_models\LLMs_fine_tuning\phi3-qlora .
!move Health_Therapist\backend\ml_models\LLMs_fine_tuning\gpt_instruction_tuning .
!move Health_Therapist\requirements.txt .
!rmdir /s /q Health_Therapist

Cloning into 'Health_Therapist'...
Access is denied.
Access is denied.
Access is denied.
The system cannot find the file specified.


        1 file(s) moved.


In [5]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
from sklearn.pipeline import Pipeline
import numpy as np
from scipy import stats
from scipy.stats import wilcoxon

In [7]:
#original here means just asking to the generic phi3 pre-trained without our fine-tuning 
#original_phi3_model_responses = pd.read_csv('phi3-qlora/original_model_responses.csv', index_col=False)

#gpt here means after asking directly to gpt 4o behave as a professional mental health assistant with prompting developed with the training dataset
#the prompt is in ...
gpt_model_responses = pd.read_csv('gpt_instruction_tuning/model_responses_from_gpt_w_instr_tun.csv', index_col=False)


# fine-tuned means after we tuned the phi3 pre-trained model with our therapists training information
fine_tuned_phi3_model_responses = pd.read_csv('phi3-qlora/model_responses.csv', index_col=False)

In [8]:
gpt_model_responses.head()

,Unnamed: 0,input,gpt_output
0,0,i want a secure relationship with someone that...,It's understandable to feel apprehensive after...
1,1,my boyfriend is in recovery from drug addictio...,Supporting a loved one in recovery can be incr...
2,2,is this something i should be worried about sh...,It's normal to feel concerned about physical c...
3,3,my friend is abusing her prescription medicine...,It's really tough to see a friend struggle wit...
4,4,i terminated my counseling relationship with a...,Ending a counseling relationship can feel unse...


In [9]:
fine_tuned_phi3_model_responses.head()

,Unnamed: 0,input,output,therapist_output
0,0,i want a secure relationship with someone that...,It sounds like you're going through a really t...,good for you on your keen awareness of your di...
1,1,my boyfriend is in recovery from drug addictio...,I'm truly sorry to hear that you're going thro...,my empathy goes out to you relationships are t...
2,2,is this something i should be worried about sh...,I'm really glad you're reaching out and sharin...,it can be tricky to figure out if a child is t...
3,3,my friend is abusing her prescription medicine...,I'm really sorry to hear that you're going thr...,first of all where did you find the pills did ...
4,4,i terminated my counseling relationship with a...,I'm really glad to hear that you're considerin...,yes your former social worker should return yo...


## Topic Modeling

In [20]:
text = fine_tuned_phi3_model_responses['input'] + ' ' + fine_tuned_phi3_model_responses['therapist_output']

In [21]:
text

0      i want a secure relationship with someone that...
1      my boyfriend is in recovery from drug addictio...
2      is this something i should be worried about sh...
3      my friend is abusing her prescription medicine...
4      i terminated my counseling relationship with a...
                             ...                        
133    i do not remember when the voices in my head s...
134    i have been hurt by a man for five years he do...
135    when my daughter is stressed about a silly thi...
136    or how to send him somewhere that can help him...
137    i have been married for years within the past ...
Length: 138, dtype: object

In [22]:
df = pd.read_csv('datasets/dataset.csv')

In [23]:
df

,Context,Response
0,i am going through some things with my feeling...,if everyone thinks you are worthless then mayb...
1,i am going through some things with my feeling...,hello and thank you for your question and seek...
2,i am going through some things with my feeling...,first thing i would suggest is getting the sle...
3,i am going through some things with my feeling...,therapy is essential for those that are feelin...
4,i am going through some things with my feeling...,i first want to let you know that you are not ...
...,...,...
2742,after first meeting the client what is the pro...,hi this is an excellent question i think that ...
2743,my boyfriend is in recovery from drug addictio...,i am sorry you have tension between you and yo...
2744,the birth mother attempted suicide several tim...,the true answer is no one can really say with ...
2745,i think adult life is making him depressed and...,how do you help yourself to believe you requir...


In [24]:
vectorizer = CountVectorizer(min_df=10, max_df=0.5, stop_words="english") # To transform the text into a numerical representation
topic_model = NMF(n_components=10)

topic_pipeline = Pipeline(
    [
        ("vectorizer", vectorizer),
        ("topic_model", topic_model),
    ]
)

In [25]:
topic_pipeline.fit(text)

Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.5, min_df=10, stop_words='english')),
                ('topic_model', NMF(n_components=10))])

In [26]:
# dictionary to map the topics

topics_map = {
    0: "Work-Life Balance",
    1: "Other",
    2: "Romantic Relationships",
    3: "Therapy and Therapist",
    4: "Negative Thoughts",
    5: "Parenthood",
    6: "People Feelings",
    7: "Anxiety and Depression",
    8: "Anxiety and Depression",
    9: "Counseling Issues"
}

In [27]:
topics = topic_pipeline.transform(text)
topics = np.argmax(topics, axis=1)
topic_names = [topics_map[topic] for topic in topics]

In [28]:
topic_names

['Romantic Relationships',
 'Romantic Relationships',
 'Parenthood',
 'People Feelings',
 'Counseling Issues',
 'Romantic Relationships',
 'Romantic Relationships',
 'Other',
 'Romantic Relationships',
 'Anxiety and Depression',
 'Parenthood',
 'Counseling Issues',
 'Anxiety and Depression',
 'Anxiety and Depression',
 'Parenthood',
 'Anxiety and Depression',
 'Negative Thoughts',
 'Anxiety and Depression',
 'Other',
 'Counseling Issues',
 'Anxiety and Depression',
 'Romantic Relationships',
 'Therapy and Therapist',
 'Romantic Relationships',
 'People Feelings',
 'Parenthood',
 'People Feelings',
 'Anxiety and Depression',
 'Therapy and Therapist',
 'People Feelings',
 'Counseling Issues',
 'Anxiety and Depression',
 'Parenthood',
 'Anxiety and Depression',
 'Counseling Issues',
 'Negative Thoughts',
 'Counseling Issues',
 'Negative Thoughts',
 'Romantic Relationships',
 'Anxiety and Depression',
 'Anxiety and Depression',
 'Counseling Issues',
 'Counseling Issues',
 'Counseling Issue

## Rouge Score with Topic Modeling

In [29]:
from modules.model_tester import ModelTester

In [31]:
rouge_tester_fine_tuned = ModelTester(fine_tuned_phi3_model_responses['therapist_output'], fine_tuned_phi3_model_responses['output'])
rouge_tester_gpt = ModelTester(fine_tuned_phi3_model_responses['therapist_output'], gpt_model_responses['gpt_output'])

In [32]:
### Here we need to do for gpt instead of the original

In [33]:
rouge_scores_fine_tuned = rouge_tester_fine_tuned.calculate_rouge_score_by_item()
rouge_scores_gpt = rouge_tester_gpt.calculate_rouge_score_by_item()

In [34]:
len(rouge_scores_fine_tuned['rouge1']['precision'])

138

In [35]:
rouge_df = pd.DataFrame()

In [36]:
for key in rouge_scores_fine_tuned.keys():
  for val in rouge_scores_fine_tuned[key].keys():
    fine_tuned_model_name = "fine-tuned_" + key + "_" + val
    gpt_model_name = "gpt_" + key + "_" + val
    rouge_df[fine_tuned_model_name] = rouge_scores_fine_tuned[key][val]
    rouge_df[gpt_model_name] = rouge_scores_gpt[key][val]

rouge_df['topic'] = topic_names
rouge_df.head()

,fine-tuned_rouge1_precision,gpt_rouge1_precision,fine-tuned_rouge1_recall,gpt_rouge1_recall,fine-tuned_rouge1_fmeasure,gpt_rouge1_fmeasure,fine-tuned_rouge2_precision,gpt_rouge2_precision,fine-tuned_rouge2_recall,gpt_rouge2_recall,fine-tuned_rouge2_fmeasure,gpt_rouge2_fmeasure,fine-tuned_rougeL_precision,gpt_rougeL_precision,fine-tuned_rougeL_recall,gpt_rougeL_recall,fine-tuned_rougeL_fmeasure,gpt_rougeL_fmeasure,topic
0,0.336066,0.434783,0.312977,0.305344,0.324111,0.358744,0.033058,0.043956,0.030769,0.030769,0.031873,0.036199,0.180328,0.206522,0.167939,0.145038,0.173913,0.170404,Romantic Relationships
1,0.445087,0.426471,0.375610,0.141463,0.407407,0.212454,0.046512,0.029851,0.039216,0.009804,0.042553,0.014760,0.179191,0.176471,0.151220,0.058537,0.164021,0.087912,Romantic Relationships
2,0.506667,0.467742,0.102981,0.078591,0.171171,0.134571,0.027027,0.016393,0.005435,0.002717,0.009050,0.004662,0.253333,0.274194,0.051491,0.046070,0.085586,0.078886,Parenthood
3,0.521739,0.483871,0.186047,0.116279,0.274286,0.187500,0.032967,0.098361,0.011673,0.023346,0.017241,0.037736,0.260870,0.258065,0.093023,0.062016,0.137143,0.100000,People Feelings
4,0.172973,0.202532,0.367816,0.183908,0.235294,0.192771,0.010870,0.000000,0.023256,0.000000,0.014815,0.000000,0.086486,0.101266,0.183908,0.091954,0.117647,0.096386,Counseling Issues


In [37]:
topics = rouge_df['topic'].unique()
topics

array(['Romantic Relationships', 'Parenthood', 'People Feelings',
       'Counseling Issues', 'Other', 'Anxiety and Depression',
       'Negative Thoughts', 'Therapy and Therapist', 'Work-Life Balance'],
      dtype=object)

In [38]:
def statistical_test_rouge(data, topics, col1, col2):
  result = []
  for topic in topics:
    data_t = data.loc[data['topic'] == topic]
    data1 = data_t[col1]
    data2 = data_t[col2]
    data1 = data1.dropna()
    data2 = data2.dropna()
    _, p_value = wilcoxon(data1, data2)
    result.append((topic, p_value, len(data1)))
  return result

In [39]:
def get_rouge_scores_df(data, topics, col1, col2):
  rouge_ttest = statistical_test_rouge(data, topics, col1, col2)
  df_result = data[['topic', col1, col2]].groupby('topic').mean()
  df_result['p_value'] = 0.0
  df_result['n'] = 0
  for result in rouge_ttest:
    df_result.loc[result[0], 'p_value'] = result[1]
    df_result.loc[result[0], 'n'] = result[2]
  return df_result


In [40]:
from IPython.display import display

In [41]:
cols = rouge_df.columns
for i in range(0, len(cols) - 1, 2):
  partial_rouge_df = get_rouge_scores_df(rouge_df, topics, cols[i], cols[i + 1])
  print(f"ROUGE scores between {cols[i]} and {cols[i + 1]}:")
  display(partial_rouge_df.style.set_properties(**{'text-align': 'center'}).set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'center')]}]
  ))
  print("\n" + "-"*80 + "\n")

ROUGE scores between fine-tuned_rouge1_precision and gpt_rouge1_precision:


,fine-tuned_rouge1_precision,gpt_rouge1_precision,p_value,n
topic,,,,
Anxiety and Depression,0.347094,0.329217,0.255613,34
Counseling Issues,0.301464,0.316707,0.135781,26
Negative Thoughts,0.342085,0.324087,0.843750,6
Other,0.369295,0.368158,1.000000,7
Parenthood,0.320798,0.353161,0.018066,15
People Feelings,0.372615,0.386544,0.325392,25
Romantic Relationships,0.375831,0.377147,0.711899,17
Therapy and Therapist,0.301859,0.337001,0.156250,6
Work-Life Balance,0.513283,0.489894,1.000000,2



--------------------------------------------------------------------------------

ROUGE scores between fine-tuned_rouge1_recall and gpt_rouge1_recall:


C:\Users\darli\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


,fine-tuned_rouge1_recall,gpt_rouge1_recall,p_value,n
topic,,,,
Anxiety and Depression,0.324480,0.183016,0.000001,34
Counseling Issues,0.316336,0.184652,0.000001,26
Negative Thoughts,0.368321,0.191678,0.031250,6
Other,0.293831,0.168132,0.015625,7
Parenthood,0.298963,0.179119,0.001225,15
People Feelings,0.313644,0.154368,0.000000,25
Romantic Relationships,0.330195,0.172661,0.000015,17
Therapy and Therapist,0.272494,0.160077,0.062500,6
Work-Life Balance,0.218200,0.114826,0.500000,2


...

C:\Users\darli\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
C:\Users\darli\anaconda3\Lib\site-packages\scipy\stats\_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


,fine-tuned_rougeL_recall,gpt_rougeL_recall,p_value,n
topic,,,,
Anxiety and Depression,0.164493,0.106712,0.000003,34
Counseling Issues,0.160944,0.104408,0.000077,26
Negative Thoughts,0.194810,0.117293,0.031250,6
Other,0.134166,0.090371,0.027708,7
Parenthood,0.146579,0.094322,0.000854,15
People Feelings,0.151902,0.086529,0.000000,25
Romantic Relationships,0.154973,0.092717,0.000015,17
Therapy and Therapist,0.136263,0.084279,0.043114,6
Work-Life Balance,0.106011,0.066511,0.500000,2



--------------------------------------------------------------------------------

ROUGE scores between fine-tuned_rougeL_fmeasure and gpt_rougeL_fmeasure:


,fine-tuned_rougeL_fmeasure,gpt_rougeL_fmeasure,p_value,n
topic,,,,
Anxiety and Depression,0.150529,0.119603,0.000017,34
Counseling Issues,0.142142,0.121934,0.001972,26
Negative Thoughts,0.145090,0.119526,0.156250,6
Other,0.131938,0.112563,0.031250,7
Parenthood,0.135697,0.115446,0.018066,15
People Feelings,0.146823,0.116474,0.000027,25
Romantic Relationships,0.156868,0.122704,0.000107,17
Therapy and Therapist,0.133616,0.108448,0.156250,6
Work-Life Balance,0.143166,0.105697,0.500000,2



--------------------------------------------------------------------------------

